### Method 01 - using cv2.addWeighted

In [7]:
import cv2
import numpy as np


# Open the webcam 
cap = cv2.VideoCapture(1)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
alpha = 0.95

# read in the first frame
ret, first_frame = cap.read()
# resize the first frame and store it in cur_frame and mhi_frame
cur_frame = mhi_frame = cv2.resize(first_frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

# Create a background
bg = np.zeros(cur_frame.shape, np.uint8)
# Set the background color to B:255, G:0, R:100
bg[:,:,0] = 255
bg[:,:,2] = 100

while True:
    ret, frame = cap.read()
    # Resize current frame
    cur_frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    
    # Make the Adaptive background
    mhi_frame = cv2.addWeighted(mhi_frame, alpha, cur_frame, 1-alpha, 0.5)
    
    # Compute the difference between current frame and the Adaptive background
    diff_frame = cv2.absdiff(cur_frame, mhi_frame)
    
    # scaled the difference between two images, makes it easier to distinguish
    diff_frame = diff_frame * 2
    
    # Let it be under 255 after scaling    
    diff_frame = np.where(diff_frame[:,:,:] > 255, 255, diff_frame)
    
    # Convert the difference in gray scale, for masking
    diff_gray = cv2.cvtColor(diff_frame, cv2.COLOR_BGR2GRAY)
    
    # to smoothen the edges
    diff_gray_guss = cv2.GaussianBlur(diff_gray,(3,3), 0)
    
    # to smoothen the edges
    diff_gray_guss = cv2.dilate(diff_gray_guss, (3,3,3))
    
    # making a binary mask
    ret ,thr = cv2.threshold(diff_gray_guss, 30, 255, cv2.THRESH_BINARY)
    
    # Generate the foreground, with blackground
    for_img = cv2.bitwise_and(cur_frame, cur_frame, mask= thr)
    
    # Generate the background mask
    thr_inv = cv2.bitwise_not(thr)
    
    # Colored the backgeound
    bg_frame = cv2.bitwise_and(bg, bg, mask=thr_inv)
    
    # Add the colored background and the foreground
    result_img = cv2.add(for_img, bg_frame)
        
    # Show the current frame
    cv2.imshow('captured image', cur_frame)
    # Show the adaptive background
    cv2.imshow('adaptive background image', mhi_frame)
    # Show the subtracted image
    cv2.imshow('subtracted image ', diff_gray)
    # Show the foreground image
    cv2.imshow('foreground image', result_img)
    
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()

### Method 02 - using BackgroundSubtractorKNN

In [3]:
import cv2
import numpy as np


# Open the webcam 
cap = cv2.VideoCapture(1)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# read in the first frame
ret, first_frame = cap.read()
# resize the first frame and store it in cur_frame and mhi_frame
cur_frame = mhi_frame = cv2.resize(first_frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

# Create a background
bg = np.zeros(cur_frame.shape, np.uint8)
# Set the background color to B:255, G:0, R:100
bg[:,:,0] = 255
bg[:,:,2] = 100

knn = cv2.createBackgroundSubtractorKNN(history = 100, dist2Threshold = 500.0, detectShadows=True)


while True:
    ret, frame = cap.read()
    # Resize current frame
    cur_frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    
    # Substract the background
    fgmask = knn.apply(cur_frame) 
    
    # Smoothen the edges
    fg_guss = cv2.GaussianBlur(fgmask, (3,3), 0)
    
    # to smoothen the edges
    fg_guss = cv2.dilate(fg_guss, (3,3,3))
    
    # making a binary mask
    ret, thr = cv2.threshold(fg_guss, 20, 255, cv2.THRESH_BINARY)
    
    # Generate the foreground, with blackground
    for_img = cv2.bitwise_and(cur_frame, cur_frame, mask= thr)
    
    # Generate the background mask
    thr_inv = cv2.bitwise_not(thr)
    
    # Colored the backgeound
    bg_frame = cv2.bitwise_and(bg, bg, mask=thr_inv)
    
    # Add the colored background and the foreground
    result_img = cv2.add(for_img, bg_frame)
    
    # Show the current frame
    cv2.imshow('captured image', cur_frame)
    # Show the adaptive background
    # cv2.imshow('adaptive background image', mhi_frame)
    # Show the subtracted image
    cv2.imshow('subtracted image ', fgmask)
    # Show the foreground image
    cv2.imshow('foreground image', result_img)
        
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()